# Road Rage: Finding the Ideal Speed Limit

### Assumptions
* Drivers want to go up to 120 km/hr.
* The average car is 5 meters long.
* Drivers want at least a number of meters equal to their speed in meters/second between them and the next car.
* Drivers will accelerate 2 m/s<sup>2</sup> up to their desired speed as long as they have room to do so.
* If another car is too close, drivers will match that car's speed until they have room again.
* If a driver would hit another car by continuing, they stop.
* Drivers will randomly (10% chance each second) slow by 2 m/s.
* This section of road is one lane going one way.
* Assume that drivers enter the road at the speed they left.
* Simulation starts with 30 cars per kilometer, evenly spaced.

## Normal Mode
We have a 1 kilometer section of road being built and do not know what the speed limit should be. This notebook simulates the 1 kilometer of road. Even though this road is not circular, the simulation treats it as such in order to generate a continuous flow of traffic.

In [17]:
import math
import matplotlib.pyplot as plt
import numpy as np
import random
from traffic_lib import *
%matplotlib inline

In [109]:
class Car:
    """
    Responsibilities:
    - Know speed (in m/s)
    - Know distance to driver ahead
    - Keep distance from driver ahead
    - Accelerate if possible
    - Match speed of those ahead if within safety zone
    - Stop if new location would result in crash (0 distance to car ahead)
    """
    
    def __init__(self, location, gap=28, speed_limit=33, start_speed=28):
        self.location = location
        self.gap = gap
        
        # speed limit is in km/hr = 1000 m / 3600 s = 1/3.6 m/s;
        self.desired_speed = speed_limit
        
        self.speed = start_speed
        self.size = 5
        self.advance = 0
        self.bumper = self.update_bumper()
            

    
    def __str__(self):
        return 'Car(location={},gap={},speed={})'.format(self.location, self.gap, self.speed)
    
    def __repr__(self):
        return self.__str__()
    
    def drive(self, car_ahead): # TODO: UPDATE GAP!
        self.advance = self.speed - car_ahead.speed
        # anticipated location of car at end of current second = car_ahead.bumper + car_ahead.speed
        # gap at end of second will be car_ahead.bumper + car_ahead.speed - self.location + self.speed
        # Closing speed
        
        # Do we calculate the anticipated location of the car ahead? We kind of are;
        # We are expecting that the car ahead will continue at their current speed


        # Test wraparound
        if car_ahead.bumper + car_ahead.speed - self.location + self.speed > self.location - car_ahead.location:
            self.gap = car_ahead.bumper - self.location
        else:
            self.gap = car_ahead.bumper + 1000 - self.location

        if self.stop(car_ahead):
            return self
        elif self.match_speed(car_ahead):
            return self
        elif self.random_slowdown(car_ahead):
            return self
        elif self.accelerate(car_ahead):
            return self

    def stop(self, car_ahead):
        if self.gap - self.advance <= 0:
            self.speed = 0
            self.location = car_ahead.bumper - 1
            if self.location < 0:
                self.location += 1000
#             self.gap = 0
            return True
        
    def match_speed(self, car_ahead):
        if self.gap - self.advance < self.speed:
            self.speed = car_ahead.speed
            self.gap = self.speed
            return True
        
    def random_slowdown(self, car_ahead):
        if random.random() < 0.1 and self.speed > 2:
            self.speed -= 2
            self.gap = (self.advance + 2)
            return True
    
    def accelerate(self, car_ahead):
        if self.speed < self.desired_speed:
            self.speed += 2
            self.gap = (self.advance - 2)
            return True

    def update_location(self):
        # What is going on? Do I want the simualtor to do this part? The simulator gives me the car ahead;
        # I should be able to do everything else.
        pass
    
    def update_bumper(self):
        if self.location >= 4:
            self.bumper = self.location - (self.size - 1)
        else:
            self.bumper = self.location - self.size + 1000
            # Note that to make the bumper location correct, we need to add 1 when adding 1000

In [67]:
class Road:
    """
    Responsibilities:
    - Hold length of road
    - Keep a list of vehicles on road
        - Initialize with number of cars
        - (1000 - sum(vehicle.size)) // len(self.vehicles)
    - Hold potential for slowdown
    
    Collaborators:
    - Car
    """
    def __init__(self, num_trucks=0):
        self.total_vehicle_space = ((30 - num_trucks) * 5) + (num_trucks * 25)
        self.initial_gap = (1000 - self.total_vehicle_space) // 30
        self.vehicles = [Car((4 + 33*n), self.initial_gap) for n in range(30 - num_trucks)]
        
        if num_trucks > 0:
            [self.vehicles.append(Truck() for _ in range(num_trucks))]

        self.vehicles[-1].gap = (1000 - self.vehicles[-1].location)
        
    def reinsert_car(self):
        if self.vehicles[-1].location < self.total_vehicle_space:
            last_car = self.vehicles.pop(-1)
            self.vehicles.insert(0, Car(last_car.location, last_car.gap))
            return True

In [105]:
class HighwaySim:
    """
    Responsibilities:
    - Have road place cars at beginning of road when they reach the end
    - Keep track of time (seconds)
    - Step through time (ticks)
        - For each car on Road, tell car behind new situation and allow it to react
        - Update locations; location - 1000 if > 1000
    - Report stats; return traffic jam status
    - Pop, Append
    
    Collaborators:
    - Car
    - Road
    """
    
    def __init__(self):
        self.road = Road()
        self.ticks = 0
        self.is_traffic = []
    
    def iterate(self):
        """
        Reverse list to process cars at the end of the road first.
        n.b., lower numbers are physically ahead of higher numbers.
        """
        for idx, v in enumerate(self.road.vehicles):
            car_ahead = self.road.vehicles[- idx - 1]
            v.drive(car_ahead)
            
            if v.location + v.speed < 1000:
                v.location = v.location + v.speed
                v.bumper = v.update_bumper()
                if v.location < car_ahead.bumper:
                    v.gap = car_ahead.bumper - v.location
                else:
                    v.gap = car_ahead.bumper + 1000 - v.location
            else:
                v.location = v.location + v.speed - 1000
                v.gap = car_ahead.bumper - v.location - 1000
                self.road.reinsert_car()
                
            
            v.gap = self.road.vehicles[- idx - 1].bumper - self.location
            
            if v.speed == 0:
                self.is_traffic.append(True)
            else:
                self.is_traffic.append(False)
    
    def run_sim(self, duration=2):
        while self.ticks < duration:
            self.iterate()
            self.ticks += 1
        if self.is_traffic.count(True) > 0:
            return True
        else:
            return False

In [106]:
sim1 = HighwaySim()

In [107]:
sim1.run_sim()

TypeError: unorderable types: int() < NoneType()

In [108]:
sim1.is_traffic.count('True')

0

In [97]:
sim1.road.vehicles

[Car(location=34,gap=-2,speed=30),
 Car(location=37,gap=28,speed=28),
 Car(location=70,gap=28,speed=28),
 Car(location=103,gap=28,speed=28),
 Car(location=136,gap=28,speed=28),
 Car(location=169,gap=28,speed=28),
 Car(location=202,gap=28,speed=28),
 Car(location=235,gap=28,speed=28),
 Car(location=268,gap=28,speed=28),
 Car(location=301,gap=28,speed=28),
 Car(location=334,gap=28,speed=28),
 Car(location=367,gap=28,speed=28),
 Car(location=400,gap=28,speed=28),
 Car(location=433,gap=28,speed=28),
 Car(location=466,gap=28,speed=28),
 Car(location=499,gap=28,speed=28),
 Car(location=532,gap=28,speed=28),
 Car(location=565,gap=28,speed=28),
 Car(location=598,gap=28,speed=28),
 Car(location=631,gap=28,speed=28),
 Car(location=664,gap=28,speed=28),
 Car(location=697,gap=28,speed=28),
 Car(location=730,gap=28,speed=28),
 Car(location=763,gap=28,speed=28),
 Car(location=796,gap=28,speed=28),
 Car(location=829,gap=28,speed=28),
 Car(location=862,gap=28,speed=28),
 Car(location=895,gap=28,speed=

In [38]:
def highway_trials(num_trials=1000):
    """
    Run num_trials of run_sim
    """
    trial_stats = []
    for _ in range(num_trials):
        trial_data = HighwaySim()
        trial_stats.append(trial_data)
    return trial_stats

In [9]:
highway_1000_data = highway_trials()

In [10]:
print(highway_1000_data.count(True))

0
